In [1]:
import numpy as np
import pandas as pd
import re
import time
import statistics

In [2]:
"""
Function:
    _parse_line

Parameters
    ----------
    filepath : str
        Filepath for file_object to be parsed

Returns
    None, None
        if not matches found.
    key, match: dictionary key and content of dictionary item
        if match found

"""

def _parse_line(line, rx_dict):
    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """

    for key, rx in rx_dict.items():
        
        match = rx.search(line)
        
        if match:
            #print(match.group(0))
            #print(key)
            return key, match
    
    # if there are no matches
    return None, None

In [3]:
"""
Function: displacement
    Calculate the DISPLACEMENT between an atom's positions (a and b) at two points in time across an MD simulation.
    
Parameters:
    a - 1x3 array containing coordinates at atom's position 1
    b - 1x3 array containing coordinates at atom's position 2
    latvectors - 3x3 array containing the real-space vectors of the supercell (a1, a2, a3)
    
Returns:
    x**2 + y**2 + z**2 - the distances between the two positions a and b 
"""

def displacement(a, b, latvectors):
    
    A = np.linalg.norm(latvectors[0])
    B = np.linalg.norm(latvectors[1])
    C = np.linalg.norm(latvectors[2])
    
    dx = abs(A*(a[0] - b[0]))
    x = min(dx, abs(A - dx))
     
    dy = abs(B*(a[1] - b[1]))
    y = min(dy, abs(B - dy))
     
    dz = abs(C*(a[2] - b[2]))
    z = min(dz, abs(C - dz))
 
    return x**2 + y**2 + z**2

In [4]:
"""

Function: get_allpositions
    Parser function to parse XDATCAR file line-by-line

Parameters:
    filepath - String containing the local path to the XDATCAR file that will be parsed.
    
Returns: 
    lattice_parameters: 3x3 array
----------- Contains the supercell real-space lattice vectors (a1,a2,a3)
    system_atoms: dictionary
----------- Contains each atom type as a key, and the number of atoms of each type as a value.
    num_MDsteps: integer
----------- Reflects the total number of MD steps as interpreted by the XDATCAR file.
    Cpos, Opos, Rpos: each Nx3 arrays
----------- Contains the positions of carbon, oxygen, and rhenium atoms (respectively) at each
        step in the MD trajectory.

"""


def get_allpositions(filepath):
    
    # Initialize array of zeros
    lattice_parameters = np.zeros((3,3), dtype=float)
    
    # open XDATCAR file and read line-by-line 
    with open(filepath, 'r') as file_object:
        
        # First line of XDATCAR file is always the system name
        system_name = file_object.readline()
        
        # Second line of XDATCAR file is always the integer 1, skip this line
        skip = file_object.readline()
        
        
        # Next three lines contain the supercell parameters. Store these parameters into variable 'lattice_parameters'.
        # Strip the whitespace off the ends of each line, 'split' the elements of the line into a list,
        # and lastly, convert the list of strings into a list of float values.
        for i in range(0,len(lattice_parameters)):
            line = file_object.readline()
            line = line.strip()
            line = line.split()
            line = list(map(float,line))
            lattice_parameters[i] = line
        
        # Sixth line contains element species.
        # Strip line, and split it so each element species is stored in a separate index of a list called 'atom_types'
        line = file_object.readline()
        line = line.strip()
        atom_types = line.split()
        
        # Seventh line contains the number of each ion specie.
        line = file_object.readline()
        line = line.strip()
        line = line.split()
        num_each_atom = list(map(int,line))
        
        # Store atom species and corresponding number of atoms in the system in a dictionary
        # for easier access later.
        system_atoms = {atom_types[i]: num_each_atom[i] for i in range(len(atom_types))}
        
        line = file_object.readline()
        
        # Counter for how many MD steps are in the XDATCAR file
        num_MDsteps = 0
        
        # initialize empty list to store all C positions across simulation
        Cpos = []
        Opos = []
        Rpos = []
        
        while line:
            
            # initialize empty list to store all C positions across simulation
            temp_Cpos = []
            temp_Opos = []
            temp_Rpos = []
            
            key, match = _parse_line(line, reg_dict)
            
            if key == 'configuration':
                num_MDsteps += 1
            
            line = file_object.readline()
            
            
            # Initialize a counter variable to 0, which increments in each while loop below.
            counter = 0
            
            # Store all carbon positions in current MD step.
            while counter != num_each_atom[0]:
                
                line = line.strip()
                line = line.split()
                coords = list(map(float,line))
                temp_Cpos.append(coords)
                counter+=1
                line = file_object.readline()
                
            # Reset counter variable to 0.
            counter = 0
            
            # Store all oxygen positions in current MD step.
            while counter != num_each_atom[1]:
                
                line = line.strip()
                line = line.split()
                coords = list(map(float,line))
                temp_Opos.append(coords)
                counter+=1
                line = file_object.readline()
                
             # Reset counter variable to 0.
            counter = 0
            
            # Store all rhenium positions in current MD step.
            while counter != num_each_atom[2]:
                
                line = line.strip()
                line = line.split()
                coords = list(map(float,line))
                temp_Rpos.append(coords)
                counter+=1
                line = file_object.readline()
            
            Cpos.append(temp_Cpos)
            Opos.append(temp_Opos)
            Rpos.append(temp_Rpos)
            
        Cpos = np.array(Cpos)
        Opos = np.array(Opos)
        Rpos = np.array(Rpos)
        
        
    return lattice_parameters, system_atoms, num_MDsteps, Cpos, Opos, Rpos

In [6]:
"""
Define regular expression to identify "Direct configuration= <integer>    " in XDATCAR file
"""

reg_dict = {'configuration': re.compile('(Direct configuration=)\s+(\d+)')}

In [29]:
start = time.time()
param, atoms, num_steps, cpos_tot, opos_tot, rpos_tot = get_allpositions('../XDATCAR')
end = time.time()
print('Time for calculation: ', end-start, 's\n')

Time for calculation:  16.07362389564514 s



In [24]:
"""
Function: Mean Squared Displacement
--------- This function calculates the mean-squared displacement for a single species in an MD simulation. Input parameters
include the supercell parameters, a 3D array (numMDsteps x numatoms x 3-coordinates) containing positions of each atom
of the particular specie for each MD step in the simulation.. The function will return a 1D array containing the MSD
at each MD step in the simulation. Therefore, if one wants the MSD as function of time, one may simply multiply the
output array by the length of the time step that was used in the MD simulation.

Parameters:
cell_param: 3x3 np array
----------- Array containing the cell parameters of the supercell.

atom_pos: MxNx3 np array
----------- Array containing all the 3 coordinate positions of the N atoms at each MD step M in the simulation, hence
the array is of size MxNx3.

Returns:
msd_list: 1D np array
----------- Array containing the MSD of the atom specie at each step in the MD simulation.
"""

def calc_msd(cell_param, atom_pos):
    
    # Store number of atoms
    num_atoms = len(atom_pos[0])
    
    # Create empty list to store MSD
    msd_list = []
    
    # Store the positions at time t=0, the first MD step
    zero_pos = atom_pos[0]
    
    # Iterate through each MD step, and at each MD step, iterate through each atom
    # For each atom, call displacement function to calculate its displacement
    
    for steps in atom_pos:
        
        # List to store displacements of each atom at the current MD step, which will then be averaged to obtain MSD
        temp_dis = []
        
        # Index counter for the atoms' initial position array 'zero_pos'
        zero_index=0
        
        for atoms in steps:
            
            temp_dis.append(displacement(atoms, zero_pos[zero_index], cell_param))
            zero_index+=1
            
        # Calculate MSD and append to MSD list for this MD step
        msd_list.append(statistics.fmean(temp_dis))
    
    return msd_list
        
    
    
            

In [31]:
c_msd = calc_msd(param, cpos_tot)
o_msd = calc_msd(param, opos_tot)
r_msd = calc_msd(param, rpos_tot)

In [34]:
c_msd=np.array(c_msd)
o_msd=np.array(o_msd)
r_msd=np.array(r_msd)

In [36]:
np.savetxt('c_msd_full.csv', c_msd)
np.savetxt('o_msd_full.csv', o_msd)
np.savetxt('r_msd_full.csv', r_msd)